In [1]:
import numpy as np
import math
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [16]:
Cost_O=1
Cost_SC=1
M=40000
S=pd.read_excel(io=r'../data/fo_result_402.xlsx',sheet_name='page_1',usecols="B:OM").values.T
L=pd.read_excel(io=r'../data/fo_result_8.xlsx',sheet_name='page_1',usecols="B:I").values.T
L=L/100
MT=1./(pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").values.flatten())
Pr=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").replace([0.6,0.66,0.72],[1.2*Cost_O,1.1*Cost_O,1*Cost_O]).values.flatten()

In [25]:
m = gp.Model("opt")
#GRB.INTEGER
u0 = m.addVar(lb=0.0,vtype=GRB.CONTINUOUS, name="u0")
v0 = m.addVar(lb=0.0,vtype=GRB.CONTINUOUS, name="v0")
w0 = m.addVar(lb=0.0,vtype=GRB.CONTINUOUS, name="w0")
xij = m.addMVar((402,24),lb=0.0,vtype=GRB.CONTINUOUS,name='xij')
zijk = m.addMVar((402,24,8),lb=0.0,vtype=GRB.CONTINUOUS,name='zijk')
u = m.addMVar(lb=0.0,shape=24,vtype=GRB.CONTINUOUS, name='u')
v = m.addMVar(lb=0.0,shape=24,vtype=GRB.CONTINUOUS, name='v')
w = m.addMVar(lb=0.0,shape=24,vtype=GRB.CONTINUOUS, name='w')
absolu=m.addMVar((402,24,8),lb=0.0,vtype=GRB.INTEGER,name='abs')

In [14]:
m.setObjectiveN(sum(Pr @ xij[:, j] for j in range(24)),index=0,name='obj1')
m.setObjectiveN(sum(L[:,j] @ zijk[i,j,:] for i in range(402) for j in range(24)),index=1,name='obj2')
he=sum(sum((1-L[:,t]) @ zijk[i,t,:] for i in range(402))-(u[t]+v[t]+w[t]) for j in range(1,25) for t in range(j))
m.setObjectiveN(Cost_SC*(24*(u0+v0+w0)+he),index=2,name='obj3')

In [ ]:
m.addConstrs((xij[:,j] <= S[:,j] for j in range(24)), name="c1")
m.addConstrs((sum(zijk[i,j,:]) == S[i,j] for i in range(402) for j in range(24)), name='c2')
#m.addConstr(sum(MT @ xij[:, j] for j in range(24)) >= 28200, name='c3')
m.addConstrs((sum(zijk[i,:,:]) <= 6000 for i in range(402)), name='c3')
m.addConstrs((absolu[:,j,k] >= zijk[:,j,k]/M for j in range(24) for k in range(8)), name='c41')
m.addConstrs((absolu[:,j,k] <= zijk[:,j,k]/M+0.999 for j in range(24) for k in range(8)), name='c42')
m.addConstrs((sum(absolu[i,j,:]) == math.ceil(S[i,j]/6000) for j in range(24) for i in range(402)), name='c4')
m.addConstrs((u[j]/0.6+v[j]/0.66+w[j]/0.72 == 28200 for j in range(24)), name='c5')
m.addConstrs(name='c6')